In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import vector
import copy
import math
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

This notebook has dimensionlesss sigma

Units in Distance are: 1Unit = 0.335*10^-9 meters
Units in time are: 1 unit = 0.35*10^-9 seconds

In [ ]:
class Simulation:
    def __init__(self,length,initial_particle_x,initial_particle_y,initial_particle_z):
        self.max_displacement = length/2 #(m)
        self.min_displacement = -length/2 #(m)
        self.time_passed = 0 #(s)
        self.sigma = 0.3345*10**-9
        self.boltzmann_constant = 1.38*10**-23
        self.argon_mass = 6.63*10**-26 #(kg)
        self.instantaneous_del_momentum_x_positive = 0 #(kgm/s)
        self.totalDelMomentum_x_positive = 0 #(kgm/s)
        self.stateSnapshots = []
        self.times = []
        self.temperatures = []
        self.pressures = []
        self.current_x = copy.copy(initial_particle_x)
        self.current_y = copy.copy(initial_particle_y)
        self.current_z = copy.copy(initial_particle_z)
        self.current_v_x = np.zeros(len(initial_particle_x))
        self.current_v_y = np.zeros(len(initial_particle_x))
        self.current_v_z = np.zeros(len(initial_particle_x))
        self.current_a_x = np.zeros(len(initial_particle_x))
        self.current_a_y = np.zeros(len(initial_particle_x))
        self.current_a_z = np.zeros(len(initial_particle_x))

        self.particles_x = [copy.copy(initial_particle_x)]
        self.particles_y = [copy.copy(initial_particle_y)]
        self.particles_z = [copy.copy(initial_particle_z)]
        self.particles_v_x = [np.zeros(len(initial_particle_x))]
        self.particles_v_y = [np.zeros(len(initial_particle_x))]
        self.particles_v_z = [np.zeros(len(initial_particle_x))]
        self.particles_a_x = [np.zeros(len(initial_particle_x))]
        self.particles_a_y = [np.zeros(len(initial_particle_x))]
        self.particles_a_z = [np.zeros(len(initial_particle_x))]


   

    def update_acceleration(self):
        #Argon parameters
        sigma = 0.3345*10**-9 #m
        epsilon = 1.735*10**-21
        argon_mass = 6.63*10**-26 #kg
        for i in range(0,len(self.current_x)):
            #Instantiatet the force vector acting upon i from all other possible particles j_n
            vec_force_i  = np.asarray([0,0,0])
            for j in range(0,len(self.current_x)):
                if (i == j):
                    continue
                r_i = np.asarray([self.current_x[i],self.current_y[i],self.current_z[i]])
                r_j = np.asarray([self.current_x[j],self.current_y[j],self.current_z[j]])
                #Considering the position vector from particle j to i
                r_ji = r_i - r_j
                r_ji_mag = math.sqrt(abs(np.dot(r_ji,r_ji)))
                if (r_ji_mag < 2):
                    continue
                r_ji_direction = r_ji/r_ji_mag
                
                try:
                    #Force eqn below from radial distance derivative of Lennard-Jones potential
                    if (r_ji_mag**13 != 0):
                        mag_force = (24*epsilon*(r_ji_mag**6-2))/(r_ji_mag**13)
                    else:
                        mag_force = 0
                    vec_force_ji = mag_force*r_ji_direction
                    vec_force_i = vec_force_i + vec_force_ji
                except:
                    continue
                # vec_force_i =  vec_force_ji

            #Finally we have spatial acceleration vector
            vec_acc_i = vec_force_i/argon_mass
            if (r_ji_mag**10 != 0):
                # print(vec_acc_i[0])
                self.current_a_x[i] = copy.copy(vec_acc_i[0])
                self.current_a_y[i] = copy.copy(vec_acc_i[1])
                self.current_a_z[i] = copy.copy(vec_acc_i[2])
            else:
                self.current_a_x[i] = 0
                self.current_a_y[i] = 0
                self.current_a_z[i] = 0
            

    def update_velocity(self,dt=0.00001):
        for i in range(0,len(self.current_x)):
            temp_acceleration = np.asarray([self.current_a_x[i],self.current_a_y[i],self.current_a_z[i]])
            temp_velocity = np.asarray([self.current_v_x[i],self.current_v_y[i],self.current_v_z[i]])
            temp_new_velocity = temp_velocity+(dt/2)*temp_acceleration
            self.current_v_x[i] = copy.copy(temp_new_velocity[0])
            self.current_v_y[i] = copy.copy(temp_new_velocity[1])
            self.current_v_z[i] = copy.copy(temp_new_velocity[2])
        # acceleration = np.asarray([self.current_a_x,self.current_a_y,self.current_a_z])
        # velocity = np.asarray([self.current_v_x,self.current_v_y,self.current_v_z])
        # velocity = velocity + (dt/2)*acceleration
        # self.current_v_x = velocity[0]
        # self.current_v_y = velocity[1]
        # self.current_v_z = velocity[2]

            
    def update_position(self,dt=0.00001):
        self.instantaneous_del_momentum_x_positive = 0
        for i in range(0,len(self.current_x)):
            position = np.asarray([self.current_x[i],self.current_y[i],self.current_z[i]])
            # print('position before')
            # print(position)
            velocity = np.asarray([self.current_v_x[i],self.current_v_y[i],self.current_v_z[i]])
            # print('del positiion')
            # print(dt*velocity)
            position = position + (dt)*velocity
            # print('position after')
            # print(position)
            #In +ve x direction:
            if (position[0] > self.max_displacement): #if a particle has passed 'tried' to pass the boundary 
                # print('triggered')
                #Place the particle at the boundary
                position[0] = self.max_displacement
                #Increment the total del_momentum in x for future momentum calculation
                self.instantaneous_del_momentum_x_positive += self.argon_mass*math.sqrt(abs(np.dot(velocity,velocity)))
                self.totalDelMomentum_x_positive += self.argon_mass*math.sqrt(abs(np.dot(velocity,velocity)))
                #Reverse its momentum in relevant direction to simulate an elastic collision
                velocity[0] = -1*velocity[0]
            #in -ve x direction:
            if(position[0] < self.min_displacement):
                #Place the particle at the boundary
                position[0] = self.min_displacement
                #We will not consider del momentum for any other directions, since principle of pressure symmetry should
                #allow us to calculate total pressure from total del momentum using one sign of one axis.
                velocity[0] = -1*velocity[0]
            #On y axis:
            if (position[1] > self.max_displacement): 
                position[1] = self.max_displacement
                velocity[1] = -1*velocity[1]
            if(position[1] < self.min_displacement):
                position[1] = self.min_displacement
                velocity[1] = -1*velocity[1]
            #On z axis:
            if (position[2] > self.max_displacement): 
                # print('Z +ve Boundary Hit')
                position[2] = self.max_displacement
                velocity[2] = -1*velocity[2]
            if(position[2] < self.min_displacement):
                position[2] = self.min_displacement
                velocity[2] = -1*velocity[2]

            self.current_x[i] = copy.copy(position[0])
            self.current_y[i] = copy.copy(position[1])
            self.current_z[i] = copy.copy(position[2])
            self.current_v_x[i] = copy.copy(velocity[0])
            self.current_v_y[i] = copy.copy(velocity[1])
            self.current_v_z[i] = copy.copy(velocity[2])
    

    def capture_state(self):
       
        self.particles_x.append(copy.copy(self.current_x))
        self.particles_y.append(copy.copy(self.current_y))
        self.particles_z.append(copy.copy(self.current_z))

        self.particles_v_x.append(copy.copy(self.current_v_x))
        self.particles_v_y.append(copy.copy(self.current_v_y))
        self.particles_v_z.append(copy.copy(self.current_v_z))

        self.particles_a_x.append(copy.copy(self.current_a_x))
        self.particles_a_y.append(copy.copy(self.current_a_y))
        self.particles_a_z.append(copy.copy(self.current_a_z))
        

    def calculate_instantaneous_pressure(self,delt):
        face_area = (self.max_displacement*2)**2
        p_x_instantaneous = (1/(face_area*delt))*self.instantaneous_del_momentum_x_positive
        pressure_instantaneous = 3*p_x_instantaneous 
        self.pressures.append(pressure_instantaneous)
        return pressure_instantaneous

    def calculate_average_pressure(self,delt):
        face_area = (self.max_displacement*2)**2*self.sigma
        total_time = delt*len(self.particles_x)*self.sigma
        p_x_average = (1/(face_area*total_time))*self.totalDelMomentum_x_positive
        presure_average = 3*p_x_average
        return presure_average

    def calculate_temperature(self):
        # Temperature related to total KE of a gas
        KE = 0
        N_particles = len(self.current_x)
        for i in range(0,N_particles):
            velocity_arr = np.asarray([self.current_v_x[i],self.current_v_y[i],self.current_v_z[i]])
            current_vel_mag = math.sqrt(abs(np.dot(velocity_arr,velocity_arr)))
            KE+= 0.5*self.argon_mass*current_vel_mag
        temperature = KE*(0.666)/(N_particles*self.boltzmann_constant)
        self.temperatures.append(copy.copy(temperature))
        return temperature

    def calculate_average_temperature(self):
        temperatures = np.asanyarray(self.temperatures)
        average_temp = np.mean(temperatures)
        return average_temp
    

    def main(self,delt = 0.00001,nSteps=100,maxTime=0,render = False):
        time=0
        self.times = [0]
    
        if (maxTime == 0):
            maxTime = nSteps*delt
        while (time <= maxTime):

            time = time+delt

            self.update_acceleration()
            self.update_velocity(dt = delt)
            self.update_position(dt = delt)
            self.capture_state()
            self.calculate_temperature()
            self.times.append(time+delt)
            # self.particles = currentState
        




Task 1 Testing

In [ ]:
sigma = 0.3345*10**-9 #m
initial_x = [0.0,7]
initial_y = [0.0,0.0]
initial_z = [0.0,0.0]

sim1 = Simulation(length= 100,initial_particle_x=initial_x,initial_particle_y=initial_y,initial_particle_z=initial_z)
sim1.main(delt=0.3)

xCoords = sim1.particles_x
xCoords = np.transpose(xCoords)
xVelCoords = sim1.particles_v_x
xVelCoords = np.transpose(xVelCoords)
times = sim1.times


In [ ]:
fig,ax = plt.subplots()
ylim = [-50,50]
ax.set_ylim(ylim)
ax.scatter(sim1.times,xCoords[0])
ax.scatter(sim1.times,xCoords[1])
np.shape(xVelCoords[0])

Task 2 Particles in a box

In [ ]:
sigma = 0.3345*10**-9 #m
length = 20
initial_x = [0,5]
initial_y = [0.0,0.0]
initial_z = [0.0,0.0]

sim2 = Simulation(length= length,initial_particle_x=initial_x,initial_particle_y=initial_y,initial_particle_z=initial_z)
sim2.main(delt=0.03,nSteps=1000)

xCoords = sim2.particles_x
xCoords = np.transpose(xCoords)
yCoords = sim2.particles_y
yCoords = np.transpose(yCoords)
zCoords = sim2.particles_z
zCoords = np.transpose(zCoords)

xVelCoords = sim2.particles_v_x
xVelCoords = np.transpose(xVelCoords)
xAccCoords = sim2.particles_a_x
xVelCoords = np.transpose(xVelCoords)

times = sim2.times


In [ ]:
fig,ax=plt.subplots()
ylim = [-length/2,length/2]
ax.set_ylim(ylim)

xTranspose = np.transpose(sim2.particles_x)
ax.scatter(sim2.times,xTranspose[0])
ax.scatter(sim2.times,xTranspose[1])


Task 3 - Investigate
Before proceeding it will be best to visualize & animate the simulation

In [ ]:
sigma = 0.3345*10**-9 #m
length = 20
initial_x = [-2,2]
initial_y = [0.0,0.0]
initial_z = [0.0,0.0]
zCoords = sim2.particles_z

sim2 = Simulation(length=length,initial_particle_x=initial_x,initial_particle_y=initial_y,initial_particle_z=initial_z)
sim2.main(delt=0.03,nSteps=1000)
xCoords = sim2.particles_x
yCoords = sim2.particles_y
zCoords = sim2.particles_z


In [ ]:

fig = plt.figure()
xlim = [-length/2,length/2]
fig.subplots_adjust(left=0, bottom=0, right=1, top=1)    
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(xlim)
ax.set_ylim(xlim)
# ax.set_zlim(xlim)

def update_points(num):
    num = 10*num
    if (num <= len(xCoords)):
        graph._offsets3d = (xCoords[num],yCoords[num],zCoords[num])
# index = 3
# ax.scatter(xs=xCoords[index],ys=yCoords[index],zs=zCoords[index])
    # return fig,ax
graph = ax.scatter(xs=xCoords[0],ys=yCoords[0],zs=zCoords[0])
anim = FuncAnimation(fig,update_points,frames=np.arange(0,len(xCoords)),repeat=False)
anim.save('test_gif.gif')


In [ ]:
fig,ax = plt.subplots()
xTranspose = np.transpose(sim2.particles_x)
ax.scatter(sim2.times,xTranspose[0])
ax.scatter(sim2.times,xTranspose[1])

In [ ]:
# box_length = 30
# sim3_initial_x = []
# sim3_initial_y = []
# sim3_initial_z = []
# for i in range(-2,2):
#     for j in range(-2,2):
#         for k in range(-2,2):
#             sim3_initial_x.append(i*5)
#             sim3_initial_y.append(j*5)
#             sim3_initial_z.append(k*5)

# sim3 = Simulation(length=box_length,initial_particle_x=sim3_initial_x,initial_particle_y=sim3_initial_y,initial_particle_z=sim3_initial_z)


In [199]:
box_length = 5
number_particles = 10
sim3_initial_x = []
sim3_initial_y = []
sim3_initial_z = []
for i in range(0,number_particles):
            sim3_initial_x.append(np.random.uniform(-box_length/2,box_length/2))
            sim3_initial_y.append(np.random.uniform(-box_length/2,box_length/2))
            sim3_initial_z.append(np.random.uniform(-box_length/2,box_length/2))

sim3 = Simulation(length=box_length,initial_particle_x=sim3_initial_x,initial_particle_y=sim3_initial_y,initial_particle_z=sim3_initial_z)


Introducing a change of unit
All lengths mentioned are no longer meters but are sigma meters
1(SM) = 0.3*10-9 meters

In [200]:
sim3 = Simulation(length=box_length,initial_particle_x=sim3_initial_x,initial_particle_y=sim3_initial_y,initial_particle_z=sim3_initial_z)
sim3.main(delt=0.01,nSteps=10)
xCoords3 = sim3.particles_x
# xCoords3 = np.transpose(xCoords)
yCoords3 = sim3.particles_y
# yCoords3 = np.transpose(yCoords)
zCoords3 = sim3.particles_z
# zCoords3 = np.transpose(zCoords)

xVelCoords3 = sim3.particles_v_x
yVelCoords3 = sim3.particles_v_y
zVelCoords3 = sim3.particles_v_z
# xVelCoords3 = np.transpose(xVelCoords)
xAccCoords3 = sim3.particles_a_x
yAccCoords3 = sim3.particles_a_y
zAccCoords3 = sim3.particles_a_z
avg_pressure = sim3.calculate_average_pressure(delt=0.01)
avg_temperature = sim3.calculate_average_temperature()

# xVelCoords3 = np.transpose(xVelCoords)


In [201]:
avg_temperature

0.031661339807012114

In [202]:
avg_pressure

3.5646913746163535e-05

In [ ]:
fig = plt.figure()
xlim = [-box_length/2-1,box_length/2+1]
fig.subplots_adjust(left=0, bottom=0, right=1, top=1)    
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(xlim)
ax.set_ylim(xlim)
# ax.set_zlim(xlim)

def update_points(num):
    num = num
    if (num <= len(xCoords)):
        graph._offsets3d = (xCoords3[num],yCoords3[num],zCoords3[num])
# index = 3
# ax.scatter(xs=xCoords[index],ys=yCoords[index],zs=zCoords[index])
    # return fig,ax
graph = ax.scatter(xs=xCoords3[0],ys=yCoords3[0],zs=zCoords3[0])
anim = FuncAnimation(fig,update_points,frames=np.arange(0,10),repeat=False)
anim.save('sim3_1st working_example.gif')


In [ ]:
fig = plt.figure()
xlim = [-box_length/2-0.5,box_length/2+0.5]
fig.subplots_adjust(left=0, bottom=0, right=1, top=1)    
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(xlim)
ax.set_ylim(xlim)
ax.set_zlim(xlim)

def update_points(num):
    num = num
    if (num <= len(xCoords)):
        # for i in range(0,number_particles):
        ax.scatter(xs=xCoords3[num],ys=yCoords3[num],zs=zCoords3[num])
# index = 3
# ax.scatter(xs=xCoords[index],ys=yCoords[index],zs=zCoords[index])
    # return fig,ax
graph = ax.scatter(xs=xCoords3[0],ys=yCoords3[0],zs=zCoords3[0])
anim = FuncAnimation(fig,update_points,frames=np.arange(0,5),repeat=False)
anim.save('3d.gif')


In [ ]:
zCoords3